In [133]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
path = 'gdrive/MyDrive/MadeProject'

Mounted at /content/gdrive/


In [ ]:
!git clone https://github.com/NVIDIA/DeepLearningExamples
!cd DeepLearningExamples/PyTorch/Recommendation/NCF/ && python3 -m pip install -r requirements.txt

In [ ]:
# https://stackoverflow.com/questions/66610378/unencryptedcookiesessionfactoryconfig-error-when-importing-apex
!git clone https://github.com/NVIDIA/apex
!cd apex && python3 -m pip install -v --disable-pip-version-check --no-cache-dir \
--global-option="--cpp_ext" --global-option="--cuda_ext" ./

In [18]:
!cp -a DeepLearningExamples/PyTorch/Recommendation/NCF/. /
!rm -rf DeepLearningExamples
!ls /

  Cloning https://github.com/NVIDIA/dllogger to /tmp/pip-install-zun4_gs6/dllogger_cb82a79f04b243d6be14c82e21558015
  Running command git clone -q https://github.com/NVIDIA/dllogger /tmp/pip-install-zun4_gs6/dllogger_cb82a79f04b243d6be14c82e21558015
  Created wheel for dllogger: filename=DLLogger-0.1.0-py3-none-any.whl size=5670 sha256=e7d2f1804542817410edba608288b7a195f1d7845627f375134f098ca17173ea
  Stored in directory: /tmp/pip-ephem-wheel-cache-zjw4rl0s/wheels/f6/0b/e6/d9566189ca2a94642aa7060bbf15c51ec8c464053945b1358b
Successfully built dllogger


In [19]:
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!mkdir -p /data/ml-1m
!mv ml-1m.zip /data/ml-1m
!ls /data/ml-1m/

--2021-11-27 09:44:50--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  4.18MB/s    in 1.4s    

2021-11-27 09:44:53 (4.18 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

ml-1m.zip


In [134]:
import pandas as pd

#interaction_path = 'interaction_by_orders.parquet'
#interaction_path = 'interaction_by_orders_small.parquet'
interaction_path = 'ncf_orders.parquet'
df = pd.read_parquet(f'{path}/{interaction_path}')
df.to_csv('/data/interactions.csv')
df.head()

,user_id,chain_id,weight
index,,,
2,63993755,51569,1
3,28610448,19783,1
5,42542701,42442,1
6,41676733,45789,1
7,7371423,45789,1


In [135]:
# !cd / && ./prepare_dataset.sh ml-1m
!cd / && mkdir -p /data/cache/interactions && python3 convert.py \
    --path /data/interactions.csv \
    --output /data/cache/interactions \
    --valid_negative=100

Loading raw data from /data/interactions.csv
Mapping original user and item IDs to new sequential IDs
(8480907, 2)
652614 681463
[     0      1      2 ... 681462 681462 681462]
[  0   1   2 ... 286 609 721]
Generating validation negatives...
100% 34073150/34073150 [00:21<00:00, 1598947.20it/s]


In [136]:
!cd / && python3 -m torch.distributed.launch --nproc_per_node=1 --use_env ncf.py --data /data/cache/interactions  --checkpoint_dir /data/checkpoints/

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
DLL 2021-11-27 13:42:20.770602 - PARAMETER data : /data/cache/interactions  feature_spec_file : feature_spec.yaml  epochs : 30  batch_size : 1048576  valid_batch_size : 1048576  factors : 64  layers : [256, 256, 128, 64]  negative_samples : 4  learning_rate : 0.0045  topk : 10  seed : None  threshold : 1.0  beta1 : 0.25  beta2 : 0.5  eps : 1e-08  dropout : 0.5  checkpoint_dir : /data/checkpoints/  load_checkpoint_path : None  mode : train  grads_accumulated : 1  amp : False  log_path : log.json  world_size : 1  distributed :

In [137]:
!cd / && python3 -m torch.distributed.launch --nproc_per_node=1 --use_env ncf.py -k 30 --data /data/cache/interactions  --mode test --load_checkpoint_path /data/checkpoints/model.pth

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
DLL 2021-11-27 13:46:01.138216 - PARAMETER data : /data/cache/interactions  feature_spec_file : feature_spec.yaml  epochs : 30  batch_size : 1048576  valid_batch_size : 1048576  factors : 64  layers : [256, 256, 128, 64]  negative_samples : 4  learning_rate : 0.0045  topk : 30  seed : None  threshold : 1.0  beta1 : 0.25  beta2 : 0.5  eps : 1e-08  dropout : 0.5  checkpoint_dir :   load_checkpoint_path : /data/checkpoints/model.pth  mode : test  grads_accumulated : 1  amp : False  log_path : log.json  world_size : 1  distribut